*https://github.com/Aubrey-Bermuda-CA/IFRS9-MODELING-CHINA*

*3.3 Function description: Read macro historical data and external prediction data, use HP filter to filter the data, and use SAR IMAX to predict macro data. This version offers more customization details.*

In [1]:
import re
import time
import itertools
import numpy as np
import pandas as pd
import pmdarima as pm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

In [2]:
# set date range
start_date = '2010-03-31'
history_date = '2024-09-30'
forecast_date = '2026-09-30'
date_index = pd.date_range(start=start_date, end=forecast_date, freq='Q')

# read macro data
macro_his_data = pd.read_csv(f'./macrofactor_historical_data.csv', index_col=[0], parse_dates=[0])
macro_fc_data = pd.read_csv(f'./macrofactor_forecast_data.csv', index_col=[0], parse_dates=[0])

# combine data
macro_data = macro_his_data.combine_first(macro_fc_data)

# Missing values ​​can be handled using interpolation. There are several methods to choose from:
# 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial', 
# 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline'
# macro_data = macro_data.interpolate(method='spline')

macro_data

,GDP_Const_Qtr,Ind_Pro_Mth,CPI_Mth,Core_CPI_Mth,PPI_Mth,FAI_Cum,Retail_Sales_Mth,Imports_Mth,Exports_Mth,M1,...,PPIRM_Mth,CGPI_Mth,70_Cities_Price_Mth,Power_Gen_Mth,Freight_Mth,Urban_Income_Cum,Urban_Exp_Cum,Nat_Housing_Index,Macro_Eco_Index,Housing_Sales_Cum
date,,,,,,,,,,,,,,,,,,,,,
2010-03-31,12.20,18.1,102.400,NaN,5.91,26.4,18.00,66.27,24.21,29.94,...,11.46,105.6,9.5,17.6,15.800000,9.8,11.01,105.89,104.0,57.7
2010-06-30,10.80,13.7,102.900,NaN,6.41,25.5,18.30,33.87,43.87,24.56,...,10.80,106.6,7.7,11.4,12.300000,10.2,9.89,105.06,102.6,25.4
2010-09-30,9.90,13.3,103.600,NaN,4.33,24.5,18.84,24.38,25.08,20.87,...,7.10,106.1,6.2,8.1,11.900000,10.5,9.32,103.52,101.9,15.9
2010-12-31,9.90,13.5,104.600,NaN,5.93,24.5,19.10,25.94,17.87,21.19,...,9.47,107.9,5.0,5.1,0.310000,11.3,9.84,101.79,103.6,18.3
2011-03-31,10.20,14.8,105.383,NaN,7.31,25.0,17.40,27.50,35.76,15.00,...,10.53,109.3,3.7,14.8,14.300000,12.3,10.69,102.98,102.5,27.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-30,4.60,5.4,100.400,0.1,-2.80,3.4,3.20,0.30,2.40,-7.40,...,-2.20,98.3,-9.0,6.0,2.315156,4.5,5.00,92.40,NaN,-22.7
2024-12-31,4.85,NaN,1.800,0.7,-2.20,3.0,6.40,11.00,NaN,-6.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-31,4.00,NaN,NaN,NaN,NaN,3.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# The method used here comes from this article ：https://www.ecb.europa.eu/pub/pdf/scpwps/ecbwp499.pdf. 
# This is just a simple attempt, not a reproduction of the article.
# As concluded in the article, hpfilter obtain more stable predictive values.

# Use Hodrick-Prescott filter to separate trend terms and periodic terms
macro_data_fitted = pd.DataFrame(index=macro_data.index)

for column in macro_data:
    y = macro_data[column].dropna()
    cycle, trend = sm.tsa.filters.hpfilter(y)
    try:
        result = STL(cycle).fit()
        seasonal = result.seasonal
        seasonal = seasonal.rename(column)
        seasonal_smooth = seasonal[abs(seasonal - seasonal.mean()) <= 2.576 * seasonal.std()] # 99% confidence level value
        fitted = trend.add(seasonal_smooth, fill_value=0)
    except:
        seasonal = pd.DataFrame(index=trend.index, columns=[column])
        cycle_smooth = cycle[abs(cycle - cycle.mean()) <= 2.576 * cycle.std()]
        fitted = trend.add(cycle_smooth, fill_value=0)
    
    # # Create a graphics object
    # fig, ax1 = plt.subplots(figsize=(10, 5))
    # ax1.plot(y.index, y, label=column, color='g')
    # ax1.plot(y.index, fitted, label='Fitted', color='b')
    # ax1.set_xlabel('Date')
    # ax1.set_ylabel('Value and Fitted')
    # ax1.legend(loc='upper left')

    # # Create a secondary axis
    # ax2 = ax1.twinx() 
    # ax2.plot(y.index, seasonal, label='Seasonal', color='gray', linestyle='--')
    # ax2.set_ylabel('Seasonal', color='k')
    # ax2.legend(loc='upper right')
    # plt.title('Original, Seasonal and Fitted (HP filter)')
    # plt.show()

    macro_data_fitted = macro_data_fitted.merge(fitted.rename(column), left_index=True, right_index=True, how='left')

macro_data_fitted.to_csv(f'./3.3_macro_data_fitted.csv')

In [4]:
# to avoid Maximum Likelihood convergence warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='statsmodels')

# macrofactor forecast with sarimax
df = macro_data_fitted
# Set the frequency
df = df.asfreq('Q-DEC')
predict_data = pd.DataFrame(index=date_index)
print(f'macrofoctor_list：{df.columns.values}')

# timer
start_t = time.perf_counter()

# loss function, You can customize your own loss function
def func_calc_rmse(y, y_pred):
    try:
        res_rmse = np.sqrt(np.mean((y[-16:-4] - y_pred[-16:-4]) ** 2))
    except:
        res_rmse = np.inf
    return res_rmse

# Define parameter range
p_values = range(2, 5)
q_values = range(2, 5)
P_values = range(0, 2)
Q_values = range(0, 2)

# fit model
for column in df:
    y = df[column].dropna()
    print(f"\n Traversing the SARIMA equations of {y.name} ...")
    # seasonal data use 4
    m = 4
    # calc d and D firt to save time
    d = pm.arima.ndiffs(y, alpha=0.05, test='adf', max_d=2)
    D = 0 # Differences in seasonal items can cause abnormal fluctuations in data
    
    # Initialize optimal parameters and maximum loss function
    best_rmse = float('inf')
    best_params = None
    
    # Traversing
    for p, q, P, Q in itertools.product(p_values, q_values, P_values, Q_values):
        try:
            time_dur0 = time.perf_counter()
            # fit sarimax model
            model = SARIMAX(y, order=(p, d, q), seasonal_order=(P, D, Q, m), enforce_stationarity=False, enforce_invertibility=False)
            results = model.fit(disp=False)
            
            # results summary
            res_summary = results.summary().as_text()
            
            # Hypothesis testing can also be customized as needed
            # Ljung-Box(Q) test: used to test whether the residual is white noise (randomness). 
            # if p_value > 0.01, the null hypothesis is accepted, indicating that the residuals are white noise.
            Ljung_Box_match = re.search(r'Prob\(Q\):\s+([\d.]+)', res_summary)
            LB_p = float(Ljung_Box_match.group(1)) if Ljung_Box_match else None
            
            # Heteroskedasticity test: Indicates whether the variance of the error term changes over time. 
            # If p-value > 0.01, the null hypothesis is accepted, the variance of the error term is independent 
            # of time and there is no heteroskedasticity.
            # Heteroskedasticity_match = re.search(r'Prob\(H\) \(two-sided\):\s+([\d.]+)', res_summary)
            # Het_p = float(Heteroskedasticity_match.group(1)) if Heteroskedasticity_match else None

            # calculate loss function
            # mse = results.mse
            y_pred = results.predict()
            rmse = func_calc_rmse(y, y_pred)
            time_dur = time.perf_counter() - time_dur0
            print(f'({p},{d},{q})({P},{D},{Q})[{m}],	Ljung-Box(Q) = {LB_p:.2f},	RMSE = {rmse:.4f},	time = {time_dur:.2f}s')
            
            # Update optimal parameters
            if (LB_p >= 0.01) & (rmse < best_rmse):
                best_rmse = rmse
                best_params = f'({p},{d},{q})({P},{D},{Q})[{m}]'
                pr_in = y_pred
                pr_out = results.forecast(steps=15)
                
        except Exception as e:
            # print("An error occurred:", e)
            continue
    
    pr = pd.DataFrame({column:pd.concat([pr_in, pr_out])})
    print(f'Best RMSE:  {best_rmse:.4f}')
    print(f'{y.name} : Best parameters : {best_params}')
    predict_data = predict_data.merge(pr, left_index=True, right_index=True, how='left')

# Output prediction results
predict_data = macro_data.combine_first(predict_data) # This does not change the value of historical data
# predict_data = macro_data_fitted.combine_first(predict_data) # If you want to use filtered historical values
predict_data.to_csv(f'./3.3_predict_data.csv')

dur = (time.perf_counter() - start_t) / 60
print(f'The prediction process took a total of {dur:,.2f} minutes.')

macrofoctor_list：['GDP_Const_Qtr' 'Ind_Pro_Mth' 'CPI_Mth' 'Core_CPI_Mth' 'PPI_Mth'
 'FAI_Cum' 'Retail_Sales_Mth' 'Imports_Mth' 'Exports_Mth' 'M1' 'M2'
 'Loans_RMB' 'Social_Financing' 'Manuf_PMI' 'Ind_Revenue_Cum' 'PPIRM_Mth'
 'CGPI_Mth' '70_Cities_Price_Mth' 'Power_Gen_Mth' 'Freight_Mth'
 'Urban_Income_Cum' 'Urban_Exp_Cum' 'Nat_Housing_Index' 'Macro_Eco_Index'
 'Housing_Sales_Cum']

 Traversing the SARIMA equations of GDP_Const_Qtr ...
(2,1,2)(0,0,0)[4],	Ljung-Box(Q) = 0.09,	RMSE = 0.5459,	time = 0.06s
(2,1,2)(0,0,1)[4],	Ljung-Box(Q) = 0.39,	RMSE = 0.5173,	time = 0.07s
(2,1,2)(1,0,0)[4],	Ljung-Box(Q) = 0.67,	RMSE = 0.4893,	time = 0.04s
(2,1,2)(1,0,1)[4],	Ljung-Box(Q) = 0.80,	RMSE = 0.4707,	time = 0.07s
(2,1,3)(0,0,0)[4],	Ljung-Box(Q) = 0.89,	RMSE = 0.4863,	time = 0.08s
(2,1,3)(0,0,1)[4],	Ljung-Box(Q) = 0.87,	RMSE = 0.4649,	time = 0.15s
(2,1,3)(1,0,0)[4],	Ljung-Box(Q) = 0.68,	RMSE = 0.4927,	time = 0.06s
(2,1,3)(1,0,1)[4],	Ljung-Box(Q) = 0.44,	RMSE = 0.4492,	time = 0.12s
(2,1,4)(0,0,0)[4

C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


(2,1,2)(1,0,1)[4],	Ljung-Box(Q) = 0.96,	RMSE = 0.3701,	time = 0.11s
(2,1,3)(0,0,0)[4],	Ljung-Box(Q) = 0.54,	RMSE = 0.4341,	time = 0.03s
(2,1,3)(0,0,1)[4],	Ljung-Box(Q) = 0.50,	RMSE = 0.4212,	time = 0.04s
(2,1,3)(1,0,0)[4],	Ljung-Box(Q) = 0.49,	RMSE = 0.3907,	time = 0.03s
(2,1,3)(1,0,1)[4],	Ljung-Box(Q) = 0.44,	RMSE = 0.4321,	time = 0.05s
(2,1,4)(0,0,0)[4],	Ljung-Box(Q) = 0.80,	RMSE = 0.4052,	time = 0.07s
(2,1,4)(1,0,0)[4],	Ljung-Box(Q) = 0.91,	RMSE = 0.3634,	time = 0.06s
(3,1,2)(0,0,0)[4],	Ljung-Box(Q) = 0.57,	RMSE = 0.3979,	time = 0.04s
(3,1,2)(0,0,1)[4],	Ljung-Box(Q) = 0.44,	RMSE = 0.3946,	time = 0.04s
(3,1,2)(1,0,0)[4],	Ljung-Box(Q) = 0.35,	RMSE = 0.4055,	time = 0.04s


C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


(3,1,2)(1,0,1)[4],	Ljung-Box(Q) = 0.33,	RMSE = 0.4117,	time = 0.07s
(3,1,3)(0,0,0)[4],	Ljung-Box(Q) = 0.55,	RMSE = 0.4358,	time = 0.04s
(3,1,3)(0,0,1)[4],	Ljung-Box(Q) = 0.52,	RMSE = 0.4487,	time = 0.05s
(3,1,3)(1,0,0)[4],	Ljung-Box(Q) = 0.95,	RMSE = 0.4037,	time = 0.06s
(3,1,3)(1,0,1)[4],	Ljung-Box(Q) = 0.45,	RMSE = 0.4384,	time = 0.06s
(3,1,4)(0,0,0)[4],	Ljung-Box(Q) = 0.86,	RMSE = 0.3845,	time = 0.06s
(3,1,4)(1,0,0)[4],	Ljung-Box(Q) = 0.87,	RMSE = 0.3617,	time = 0.11s
(4,1,2)(0,0,0)[4],	Ljung-Box(Q) = 0.64,	RMSE = 0.4336,	time = 0.07s


...

(2,0,3)(1,0,0)[4],	Ljung-Box(Q) = 0.88,	RMSE = 4.7185,	time = 0.09s
(2,0,3)(1,0,1)[4],	Ljung-Box(Q) = 0.85,	RMSE = 4.2295,	time = 0.10s
(2,0,4)(0,0,0)[4],	Ljung-Box(Q) = 0.83,	RMSE = 4.6108,	time = 0.08s


C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


(2,0,4)(1,0,0)[4],	Ljung-Box(Q) = 0.95,	RMSE = 4.4859,	time = 0.07s
(3,0,2)(0,0,0)[4],	Ljung-Box(Q) = 0.78,	RMSE = 4.8932,	time = 0.08s
(3,0,2)(0,0,1)[4],	Ljung-Box(Q) = 0.99,	RMSE = 4.4614,	time = 0.06s
(3,0,2)(1,0,0)[4],	Ljung-Box(Q) = 0.93,	RMSE = 4.6238,	time = 0.10s
(3,0,2)(1,0,1)[4],	Ljung-Box(Q) = 0.96,	RMSE = 4.2382,	time = 0.11s
(3,0,3)(0,0,0)[4],	Ljung-Box(Q) = 0.69,	RMSE = 5.0217,	time = 0.08s
(3,0,3)(0,0,1)[4],	Ljung-Box(Q) = 0.91,	RMSE = 4.4332,	time = 0.11s
(3,0,3)(1,0,0)[4],	Ljung-Box(Q) = 0.80,	RMSE = 4.6739,	time = 0.09s
(3,0,3)(1,0,1)[4],	Ljung-Box(Q) = 0.97,	RMSE = 3.8175,	time = 0.11s


C:\Program Files\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


(3,0,4)(0,0,0)[4],	Ljung-Box(Q) = 0.80,	RMSE = 4.5831,	time = 0.09s
(3,0,4)(1,0,0)[4],	Ljung-Box(Q) = 0.89,	RMSE = 4.4578,	time = 0.10s
(4,0,2)(0,0,0)[4],	Ljung-Box(Q) = 0.38,	RMSE = 4.7278,	time = 0.05s
(4,0,2)(0,0,1)[4],	Ljung-Box(Q) = 0.97,	RMSE = 4.4426,	time = 0.09s
(4,0,3)(0,0,0)[4],	Ljung-Box(Q) = 0.71,	RMSE = 4.6656,	time = 0.08s
(4,0,3)(0,0,1)[4],	Ljung-Box(Q) = 0.96,	RMSE = 4.3642,	time = 0.10s
(4,0,4)(0,0,0)[4],	Ljung-Box(Q) = 0.95,	RMSE = 4.5353,	time = 0.09s
Best RMSE:  3.8175
Freight_Mth : Best parameters : (3,0,3)(1,0,1)[4]

 Traversing the SARIMA equations of Urban_Income_Cum ...
(2,1,2)(0,0,0)[4],	Ljung-Box(Q) = 0.38,	RMSE = 0.1652,	time = 0.06s
(2,1,2)(0,0,1)[4],	Ljung-Box(Q) = 0.20,	RMSE = 0.1453,	time = 0.09s
(2,1,2)(1,0,0)[4],	Ljung-Box(Q) = 0.78,	RMSE = 0.1074,	time = 0.06s
(2,1,2)(1,0,1)[4],	Ljung-Box(Q) = 0.95,	RMSE = 0.1099,	time = 0.07s
(2,1,3)(0,0,0)[4],	Ljung-Box(Q) = 0.75,	RMSE = 0.1443,	time = 0.06s
(2,1,3)(0,0,1)[4],	Ljung-Box(Q) = 0.02,	RMSE = 0.1643,	ti